In [1]:
#ProgramName: Daily-06-UpdateMasterDerivedChangingState
#Purpose: Weekly Update Master Catalog - Routine Maintenance
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the metadata captured

import os, sys, argparse, csv, pyodbc, sql, time, datetime
import sqlalchemy as db

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

now01 = datetime.datetime.now()

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [2]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running
# ===================================================================================================================
pn = r'eeeeeeeeee'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
an = r'Master\Process'
bn = r'Master\Reports'

sn = r'Master\sql'
tn = r'config'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathMP = os.path.join(parentDirectory, programDirectory, an)
pathMR = os.path.join(parentDirectory, programDirectory, bn)
pathMS = os.path.join(parentDirectory, programDirectory, sn)
pathMT = os.path.join(parentDirectory, programDirectory, tn)

procpath = pathMP
csvpath = pathMR
sqlpath = pathMS
configpath = pathMT

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMP):
        os.makedirs(pathMP, mode)
        print('Program Directory subfolder has been created: ', pathMP)
    else:
        print('Directory:', pathMP, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMR):
        os.makedirs(pathMR, mode)
        print('Program Directory subfolder has been created: ', pathMR)
    else:
        print('Directory:', pathMR, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMS):
        os.makedirs(pathMS, mode)
        print('Program Directory subfolder has been created: ', pathMS)
    else:
        print('Directory:', pathMS, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMT):
        os.makedirs(pathMT, mode)
        print('Program Directory subfolder has been created: ', pathMT)
    else:
        print('Central Config Directory:', pathMT, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass


Parent Directory is:  C:\Users\e201873842\Documents\Py
Directory: Daily >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Master\Process >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Master\Reports >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\Master\sql >>>>> Note ---- this folder already exists <<<<<
Central Config Directory: C:\Users\e201873842\Documents\Py\Daily\config >>>>> Note ---- this folder already exists <<<<<


In [3]:
#Set up table - remove previous records 
#Comment out if you want to track changes in table by the TodaysDate column
#TypeError: 'NoneType' object is not iterable --- Solution is: SET ANSI_WARNINGS OFF

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor.execute(''' 
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF EXISTS(SELECT 1 FROM dbo.MasterPRODChangeTracking)
BEGIN
   DELETE FROM Greg.dbo.MasterPRODChangeTracking WHERE [TodaysDate] > '19000101';
END
'''
,)
conn.commit()

conn.close()

now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking cleaned out - step complete ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking cleaned out - step complete *** 2021-06-14 11:23:40


In [4]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountApplicationData]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:41


In [5]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountAssessment]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:42


In [6]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountControl]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:43


In [7]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountDownstreamFeeds]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:44


In [8]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountDownstreamFeedsV15]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:45


In [9]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountGCPS_Operations]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:45


In [10]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountGSDR]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:47


In [11]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountGSDR_Synergy]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:48


In [12]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountGSDR_Synergy_Temp]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:48


In [13]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountGSDR_Temp]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:50


In [14]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountODS_WebApps]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:51


In [15]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountPPROD]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:51


In [16]:
#Load the MasterPRChangeTracking data into the staging table
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

df = pd.read_sql_query(''' 
SELECT CONVERT(char(8), GetDate(),112) as [TodaysDate]
,[DatabaseName]
,[SchemaName]
,[TableName]
,AVG(CAST(RecordCount as bigint)) AS AVGRC
,AVG(CAST(Ddifference as bigint)) AS AVGDD
,SUM(CAST(RecordCount as bigint)) AS SUMRC
,SUM(CAST(Ddifference as bigint)) AS SUMDD
FROM [Greg].[dbo].[tableRowCountPredictive_Analytics]
GROUP BY [TableName],[SchemaName],[DatabaseName]   
ORDER BY 4,5,6,7,3,2,1
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

cursor = conn.cursor()
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MasterPRODChangeTracking (TodaysDate,DatabaseName,SchemaName,TableName,AVGRC,AVGDD,SUMRC,SUMDD)
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                row.TodaysDate,
                row.DatabaseName,
                row.SchemaName,
                row.TableName,
                row.AVGRC,
                row.AVGDD,
                row.SUMRC,
                row.SUMDD                   
                )
conn.commit()
conn.close()
#print(df)
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.MasterPRODChangeTracking status collected *** 2021-06-14 11:23:52


##################################################################################################################
Note: This seperates the collection activities from the update activities
##################################################################################################################

In [17]:
#This next series of notebook blocks 'updates' the Master2T(testing) table 

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to zeroStatic

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master2T] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC = 0) 
AND (S.AVGDD = 0) 
AND (S.SUMRC = 0) 
AND (S.SUMDD = 0) THEN
UPDATE SET [DerivedStaticChging]='zeroStatic';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.Master change status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today *** 2021-06-14 11:23:52


In [18]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master2T] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0)
AND (S.AVGDD = 0) 
AND (S.SUMRC > 0) 
AND (S.SUMDD = 0) THEN
UPDATE SET [DerivedStaticChging]='Static';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.Master change status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today *** 2021-06-14 11:23:52


In [19]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master2T] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0 OR S.AVGRC < 0) 
AND (S.AVGDD > 0 OR S.AVGDD < 0) 
AND S.SUMRC > 0 
AND (S.SUMDD > 0 OR S.SUMDD < 0) THEN
UPDATE SET [DerivedStaticChging]='Chg-d';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.Master change status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today *** 2021-06-14 11:23:52


In [20]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the (Master2T) table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master2T] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0 AND S.AVGRC < 100) 
AND (S.AVGDD > 0 OR S.AVGDD < 0) 
AND (S.SUMRC > 0)
AND (S.SUMDD > 0 OR S.SUMDD < 100/2 OR S.SUMDD < 0 OR S.SUMDD = 1) THEN
UPDATE SET [DerivedStaticChging]='Chg-p';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.Master change status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today *** 2021-06-14 11:23:52


In [21]:
#ProgrammingError: ('42000', "[42000] 
#[Microsoft][ODBC SQL Server Driver][SQL Server]
#An action of type 'UPDATE' is not allowed in the 'WHEN NOT MATCHED' clause of a MERGE statement. 
#(10710) (SQLExecDirectW)")

#Solution: reverse the statement and match by source updating the specific target column with a value
    
    

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'Assessment' AND SchemaName LIKE 'dbo' AND TableName LIKE 'ASSESS_COG_RAW_BKP';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_AllGrades';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_AwardsIssued';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_CLOE';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_EnrollmentsandWithdrawals';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_FinalGrades';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_ImpersonatedSessionHistory';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_InstructorUsage';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_bds_GradeSchemeRanges_temp';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_bds_QuizUserAnswerResponses_temp';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'TMP_GOC_A';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'TMP_GOC_B';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'EdCloud_Import_BM_Items';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_AD_DATA';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_STU_EMAIL_GSDR';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_STU_EMAIL_SYN';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_TBL';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'tmp_test_tbl';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'GSDR' AND SchemaName LIKE 'dbo'  AND TableName LIKE 'kstemp1';
UPDATE [Greg].[dbo].[Master2T] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'GSDR' AND SchemaName LIKE 'dbo'  AND TableName LIKE 'srfile';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.Master change status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

#Notes:Need to find a simplier way to Merge and update Source for tables that are dropped from production
#Using the WHEN NOT and then update source to 'deleted' in the Derived column has not worked yet.

#SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
#SET NOCOUNT ON
#SET ANSI_WARNINGS OFF
#MERGE [Greg].[dbo].[Master2T] T
#USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
#WHEN NOT MATCHED
#THEN
#UPDATE SET T.[DerivedStaticChging]='Deleted';

#WHEN NOT MATCHED BY SOURCE AND LocationID =6
#THEN UPDATE SET LocationName ='Test';

 *** Note:  database table Greg.dbo.Master change status updated for today *** 2021-06-14 11:23:52


#------------------------------------------------------------------------------------------------------------------
#Seperate Master2T validation table with the core Master table
#------------------------------------------------------------------------------------------------------------------

In [22]:
#This next series of notebook 'updates' the Master(Main) table 
now01 = datetime.datetime.now()
print('Begin: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to zeroStatic

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC = 0) 
AND (S.AVGDD = 0) 
AND (S.SUMRC = 0) 
AND (S.SUMDD = 0) THEN
UPDATE SET [DerivedStaticChging]='zeroStatic';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master zeroStatic updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

Begin:  2021-06-14 11:23:52
 *** Note:Greg.dbo.Master zeroStatic updated for today *** 2021-06-14 11:23:52


In [23]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGDD = 0) THEN
UPDATE SET [DerivedStaticChging]='Static';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master static updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:Greg.dbo.Master static updated for today *** 2021-06-14 11:23:52


In [24]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0)
AND (S.AVGDD = 0) 
AND (S.SUMRC > 0) 
AND (S.SUMDD = 0) OR (S.SUMDD = -1) OR (S.SUMDD = 1 AND S.SUMDD < 6) THEN
UPDATE SET [DerivedStaticChging]='Static';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master static updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:Greg.dbo.Master static updated for today *** 2021-06-14 11:23:52


In [25]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0 OR S.AVGRC < 0) 
AND (S.AVGDD > 0 OR S.AVGDD < 0) 
AND S.SUMRC > 0 
AND (S.SUMDD > 0 OR S.SUMDD < 0) THEN
UPDATE SET [DerivedStaticChging]='Chg-d';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master Chg-d status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:Greg.dbo.Master Chg-d status updated for today *** 2021-06-14 11:23:52


In [26]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0 AND S.AVGRC < 1000) 
AND (S.AVGDD > 0 OR S.AVGDD < 0) 
AND (S.SUMRC > 0)
AND (S.SUMDD > 0 OR S.SUMDD < 100/2) THEN
UPDATE SET [DerivedStaticChging]='Chg-p';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master Chg-p(periodic) updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:Greg.dbo.Master Chg-p(periodic) updated for today *** 2021-06-14 11:23:52


In [27]:
#ProgrammingError: ('42000', "[42000] 
#[Microsoft][ODBC SQL Server Driver][SQL Server]
#An action of type 'UPDATE' is not allowed in the 'WHEN NOT MATCHED' clause of a MERGE statement. 
#(10710) (SQLExecDirectW)")

#Solution: reverse the statement and match by source updating the specific target column with a value

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'Assessment' AND SchemaName LIKE 'dbo' AND TableName LIKE 'ASSESS_COG_RAW_BKP';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_AllGrades';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_AwardsIssued';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_CLOE';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_EnrollmentsandWithdrawals';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_FinalGrades';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_ImpersonatedSessionHistory';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_InstructorUsage';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_bds_GradeSchemeRanges_temp';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_bds_QuizUserAnswerResponses_temp';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'TMP_GOC_A';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'TMP_GOC_B';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'EdCloud_Import_BM_Items';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_AD_DATA';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_STU_EMAIL_GSDR';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_STU_EMAIL_SYN';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_TBL';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'tmp_test_tbl';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'GSDR' AND SchemaName LIKE 'dbo'  AND TableName LIKE 'kstemp1';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted Static' WHERE DatabaseName LIKE 'GSDR' AND SchemaName LIKE 'dbo'  AND TableName LIKE 'srfile';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master updated deleted status for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:Greg.dbo.Master updated deleted status for today *** 2021-06-14 11:23:52


In [28]:
#------------------------------------------------------------------------------------------------------------------
#Seperate Master2T validation table with the core Master table
#------------------------------------------------------------------------------------------------------------------

In [29]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Clear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

# Update the 'Master' table with any changes found to zeroStatic, Static, Changing, Retired, Deleted status
# Using T = Target and S = Source ---- update the TARGET (Master) table with those tables removed from production
# Test logic on Master2T first then change to 'Master'
#################### Need 'when not matched' logic to set field in Master as deleted ############################
    
cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
SELECT T.DatabaseName, T.SchemaName, T.TableName, T.DerivedStaticChging INTO #tempGreg 
FROM (SELECT DatabaseName, SchemaName, TableName, DerivedStaticChging FROM Greg.dbo.Master) AS T
  left outer join Greg.dbo.MasterPRODCatalog
    ON T.DatabaseName = MasterPRODCatalog.DatabaseName 
   AND T.SchemaName = MasterPRODCatalog.SchemaName
   AND T.TableName = MasterPRODCatalog.TableName
 WHERE MasterPRODCatalog.DatabaseName IS NULL
 AND MasterPRODCatalog.SchemaName IS NULL
 AND MasterPRODCatalog.TableName IS NULL
MERGE [Greg].[dbo].[Master] T
USING #tempGreg S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED THEN UPDATE SET T.DerivedStaticChging='Deleted';  
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:  database table Greg.dbo.Master flagged with update for deleted tablenames ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master flagged with update for deleted tablenames *** 2021-06-14 11:23:52


In [30]:
#Connect to SQL Server

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

cursor.execute(''' 
SELECT DatabaseName, SchemaName, TableName, DerivedStaticChging FROM [Greg].[dbo].[Master] WHERE DerivedStaticChging LIKE 'Deleted'
'''
)
for row in cursor:
    print('row = %r' % (row,))
    
conn.close()

row = ('Assessment', 'dbo', 'ASSESS_COG_RAW_BKP', 'Deleted')
row = ('Assessment', 'dbo', 'CUA_TestCourseNos', 'Deleted')
row = ('Assessment', 'dbo', 'DA_Cluster_Schools', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_AllGrades', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_AwardsIssued', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_CLOE', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_EnrollmentsandWithdrawals', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_FinalGrades', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_ImpersonatedSessionHistory', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_ads_InstructorUsage', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_bds_GradeSchemeRanges_temp', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'import_bds_QuizUserAnswerResponses_temp', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'TMP_GOC_A', 'Deleted')
row = ('DownStreamFeeds', 'd2l', 'TMP_GOC_B', 'Deleted')
row = ('DownStreamFee

In [31]:
now01 = datetime.datetime.now()
print(' Update Master & Master Complete: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

 Update Master & Master Complete:  2021-06-14 11:23:53
